## churn.all

In [1]:
import pandas as pd

#放在不同資料夾用絕對路徑
data = pd.read_csv('churn.all.csv')

#刪除不要的欄位
data.drop(['id'], inplace=True, axis=1)
data.drop(['phone_number'], inplace=True, axis=1)

#將目標屬性轉為數字
import numpy as np
data['Churn'] = np.where(data['Churn'] == ' False', 0, 1)

#分割輸入屬性及目標屬性

#輸入屬性
x_ = data.drop(['Churn'], axis=1)#同上方法刪除行

#目標屬性
y_ = data['Churn']

#去除錯誤值及空值填補
from sklearn.pipeline import Pipeline
from data_preprocessing_pipeline import *
from feature_engineering import *

pipe = Pipeline(
       [
        ('define_column_type', define_column_type()),
        ('scaler', scaler()),
        ('dummies', dummies()),
        #('filter', feature_selection(method = 'anova_kf', p_threshold = 1)),
        #('sfs',feature_selection(method = 'sfs', sfs_direction = direction, sfs_scoring = scoring, sfs_cv = cv, sfs_estimator = estimator))
       ])

x_ = pipe.fit_transform(x_, y_)
print(x_.shape)

2021 15:23:46 INFO Fit Columns Type Already Success!!!
2021 15:23:46 INFO Transform Columns Type Success!!!
2021 15:23:46 INFO scaler Success!!!
2021 15:23:46 INFO dummies Success!!!


(5000, 87)


## bank

import pandas as pd

#放在不同資料夾用絕對路徑
data = pd.read_csv('bank.csv')

#刪除ID
#data.drop(['id'], inplace=True, axis=1)

#將目標屬性轉為數字
import numpy as np
data['y'] = np.where(data['y'] == 'no', 0, 1)

#分割輸入屬性及目標屬性

#輸入屬性
x_ = data.drop(['y'], axis=1)#同上方法刪除行
#x_ = data.drop(['phone_number'], axis=1)#同上方法刪除行

#目標屬性
y_ = data['y']

#去除錯誤值及空值填補
from sklearn.pipeline import Pipeline
from data_preprocessing_pipeline import *
from feature_engineering import *

pipe = Pipeline(
       [
        ('Wrong_Value',wrong_value_fillna(wrong_value=['unknown'])),
        ('Null_Value', xgb_fill()),
        ('define_column_type', define_column_type()),
        ('scaler', scaler()),
        ('dummies', dummies()),
       ])

x_ = pipe.fit_transform(x_, y_)
print(x_.shape)

## WPBC

import pandas as pd

#放在不同資料夾用絕對路徑
wpbc = pd.read_csv('wpbc.csv', header=None)

#刪除ID
wpbc.drop([0], inplace=True, axis=1)

#將目標屬性轉為數字
import numpy as np
wpbc[1] = np.where(wpbc[1] == 'N', 1, 0)

#分割輸入屬性及目標屬性

#輸入屬性
x_ = wpbc.drop([1], axis=1)#同上方法刪除行

#目標屬性
y_ = wpbc[1]

#去除錯誤值及空值填補
from sklearn.pipeline import Pipeline
from data_preprocessing_pipeline import *
from feature_engineering import *

pipe = Pipeline(
       [
         ('Wrong_Value',wrong_value_fillna(wrong_value=['?'])),
         ('column_type', define_column_type()),
         #('Outlier', fix_outlier(how='quartile')), 
         
         ('Null_Value', xgb_fill()),
         #('num_log', num_log()),
         
         ('Wrong_Value2',wrong_value_fillna(wrong_value=[np.nan, float('-inf')])),
         ('Null_Value2', xgb_fill()),
  
       ])

x_ = pipe.fit_transform(x_, y_)
print(x_.shape)

## 用產生資料測試

from sklearn.datasets import make_classification
import pandas as pd

x_, y_ = make_classification(n_samples=100, n_features=100,
                             n_informative=50, n_redundant=25,
                             n_classes=2, random_state=0, shuffle=True)

x_ = pd.DataFrame(x_)
y_ = pd.DataFrame(y_)

## 將index轉換為欄位名稱

In [2]:
def index_to_colname(indexs, oridata):
    colname = []
    for i in indexs:
        colname.append(list(x_.columns)[i])
    
    return(colname)

## 輸入模型將屬性重要性轉為index輸出

In [3]:
def importance_to_index(clf, prob):
    importances = list(clf.feature_importances_)
    s = importances
    s2 = sorted(range(len(s)), key=lambda k: s[k], reverse=True)
    return s2[:int(len(s2)*prob)]

x_ = x_[:1000]
y_ = y_[:1000]

## 交叉驗證AUC

In [4]:
#計算AUC函數
def get_auc(model, x, y):
    
    '''
    model : 用甚麼方式建模
    x : 輸入屬性
    y : 目標屬性
    '''
    import numpy as np
    from sklearn import metrics
    
    #建模
    #model.fit(x, y)

    #用訓練資料測試
    fpr, tpr, thresholds = metrics.roc_curve(y, model.predict(x))
    auc = metrics.auc(fpr, tpr)

    print('這次的AUC : ', auc,'\n')
    return auc

In [5]:
#計算f1函數
def get_f1(model, x, y):
    
    '''
    model : 用甚麼方式建模
    x : 輸入屬性
    y : 目標屬性
    '''
    import numpy as np
    from sklearn.metrics import f1_score
    
    #建模
    #model.fit(x, y)

    #用訓練資料測試
    f1 = f1_score(y, model.predict(x), average='weighted')

    print('這次的f1 : ', f1,'\n')
    return f1

from sklearn.metrics import f1_score

y_true = [0, 1, 2, 0, 1, 2]
y_pred = [0, 2, 1, 0, 0, 1]
print(f1_score(y_true, y_pred, average='macro'))  # 0.26666666666666666
print(f1_score(y_true, y_pred, average='micro'))  # 0.3333333333333333
print(f1_score(y_true, y_pred, average='weighted'))  # 0.26666666666666666
print(f1_score(y_true, y_pred, average=None))  # [0.8 0.  0. ]


In [16]:
from sklearn.ensemble import AdaBoostClassifierZe
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import numpy as np


#交叉驗證，分層隨機抽樣
from sklearn.model_selection import StratifiedKFold
import random as rd

'''
n_splits：Ｎ折交叉驗證
random_state：設定隨機種子（int）
shuffle：是否要把輸入資料順序打亂
（shuffle=True時random_state才有作用）
'''
rand = rd.randint(1, 10) #產生交叉驗證抽樣隨機種子
skf = StratifiedKFold(n_splits=10, random_state = rand, shuffle=True)

'''
x_ : 輸入屬性
y_ : 目標屬性

x_train：訓練資料輸入屬性
y_train：訓練資料目標屬性

x_test：測試資料輸入屬性
y_test：測試資料目標屬性
'''

ans_auc = [] #儲存每次交叉驗證的AUC
ans_f1 = [] #儲存每次交叉驗證的f1

for train_index, test_index in skf.split(x_, y_):
    #print("TRAIN:", train_index, "TEST:", test_index,'\n')
    x_train, x_test = x_.iloc[train_index], x_.iloc[test_index]
    y_train, y_test = y_.iloc[train_index], y_.iloc[test_index]
    
    x_train = pd.DataFrame(x_train)
    y_train = pd.DataFrame(y_train)

    x_test = pd.DataFrame(x_test)
    y_test = pd.DataFrame(y_test)   
    
    
    
    #clf = AdaBoostClassifier(n_estimators=100, random_state=0, algorithm='SAMME')
    #clf = GaussianNB()
    clf = AdaBoostClassifierZe(base_estimator=GaussianNB(), n_estimators=100, random_state=0, algorithm='SAMME', estimator_error_calc = 8, fs_enable=True)
    #clf = AdaBoostClassifier(base_estimator=SVC(probability=True), n_estimators=100, random_state=0, algorithm='SAMME')
    clf = clf.fit(x_train, y_train)
    
    clf.fit(x_train, y_train)
    
    ans_f1.append(get_f1(clf,x_test,y_test)) #取得用測試資料測試的f1
    ans_auc.append(get_auc(clf,x_test,y_test)) #取得用測試資料測試的AUC
    
#print(np.mean(ans_auc)) #將十折交叉驗證AUC加總平均

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3709: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())
2021 15:25:19 INFO feature select success!
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3709: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())
2021 15:25:19 INFO feature select success!
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3709: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_o

After select attr:  [39, 75, 14, 69, 71, 66, 36, 51, 22, 21, 17, 16, 47, 70, 54, 65, 64, 26, 72, 13, 11, 23, 68, 12, 59, 29, 8, 10, 15, 7, 5, 1, 84, 85, 20, 19, 18, 4, 2, 77, 78]
Number of features selected:  41
After select attr:  [30, 53, 48, 45, 6, 57, 62, 33, 46, 25, 80, 50, 27, 56, 63, 31, 58, 81, 37, 73, 0, 60, 52, 9, 39, 75, 14, 69, 71, 66, 36, 51, 22, 21, 17, 16, 47, 70, 54, 65, 64, 26, 72, 13, 11, 23, 68, 12, 59, 29, 8, 10, 15, 7, 5, 1, 84, 85, 20, 19, 18, 4, 2, 77, 78]
Number of features selected:  65
X1
弱分類器輸入資料維度 :  41
弱分類器錯誤率: 0.1337777777777778
SAMME弱分類器權重: 1.8679614347586848
After select attr:  [47, 43, 33, 73, 12, 36, 80, 25, 53, 81, 51, 65, 21, 42, 70, 50, 59, 64, 41, 66, 0, 61, 45, 19, 56, 8, 10, 11, 13, 29, 18, 7, 5, 1, 84, 85, 77, 78, 4, 2]
Number of features selected:  40
After select attr:  [16, 60, 49, 58, 71, 6, 22, 28, 74, 35, 62, 67, 17, 26, 75, 54, 44, 15, 72, 32, 30, 34, 20, 47, 43, 33, 73, 12, 36, 80, 25, 53, 81, 51, 65, 21, 42, 70, 50, 59, 64, 41, 66, 0, 6

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3709: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())
2021 15:25:19 INFO feature select success!
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3709: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())
2021 15:25:19 INFO feature select success!
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3709: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())
2021 15:25:19 INFO feature select success!
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3709: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warni

After select attr:  [63, 31, 41, 9, 33, 80, 27, 67, 61, 38, 69, 17, 70, 6, 1, 64, 52, 55, 26, 16, 71, 47, 49, 20, 23, 56, 42, 82, 22, 45, 59, 58, 39, 81, 68, 54, 74, 19, 3, 29, 84, 85, 72, 15, 50, 75, 14, 46, 62, 77, 78, 43, 37, 10, 8, 40, 48, 65, 11, 13, 51, 12, 18, 5, 7, 4, 2]
Number of features selected:  67
X1
弱分類器輸入資料維度 :  40
弱分類器錯誤率: 0.4848015407381052
SAMME弱分類器權重: 0.06081257138237399
After select attr:  [71, 82, 47, 64, 67, 14, 17, 75, 34, 77, 78, 61, 65, 56, 13, 11, 12, 29, 37, 15, 60, 54, 72, 62, 40, 18, 46, 58, 43, 1, 51, 48, 68, 10, 8, 84, 85, 7, 5, 4, 2]
Number of features selected:  41
After select attr:  [9, 0, 38, 55, 28, 69, 35, 27, 3, 16, 45, 74, 42, 52, 70, 22, 39, 36, 31, 63, 81, 6, 57, 59, 41, 71, 82, 47, 64, 67, 14, 17, 75, 34, 77, 78, 61, 65, 56, 13, 11, 12, 29, 37, 15, 60, 54, 72, 62, 40, 18, 46, 58, 43, 1, 51, 48, 68, 10, 8, 84, 85, 7, 5, 4, 2]
Number of features selected:  66
X1
弱分類器輸入資料維度 :  41
弱分類器錯誤率: 0.3722526342011262
SAMME弱分類器權重: 0.5225650538013863
After 

2021 15:25:19 INFO feature select success!
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3709: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())
2021 15:25:19 INFO feature select success!
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3709: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())
2021 15:25:19 INFO feature select success!
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3709: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())
2021 15:25:19 INFO feature select success!
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3709: F_onewayConstantInputWarning: Each of the input arrays is constant;the F s

...

2021 15:26:28 INFO feature select success!
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3709: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())
2021 15:26:28 INFO feature select success!
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3709: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())
2021 15:26:28 INFO feature select success!
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3709: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())
2021 15:26:28 INFO feature select success!
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3709: F_onewayConstantInputWarning: Each of the input arrays is constant;the F s

After select attr:  [4, 2, 73, 45, 15, 48, 34, 12, 42, 59]
Number of features selected:  10
After select attr:  [36, 75, 57, 37, 3, 38, 60, 56, 65, 49, 54, 53, 23, 5, 7, 32, 16, 44, 6, 28, 43, 39, 58, 64, 61, 46, 55, 20, 67, 22, 18, 68, 30, 21, 25, 84, 85, 9, 4, 2, 73, 45, 15, 48, 34, 12, 42, 59]
Number of features selected:  48
X1
弱分類器輸入資料維度 :  10
弱分類器錯誤率: 0.48641799780801886
SAMME弱分類器權重: 0.05434137726084949
After select attr:  [0, 81, 3, 5, 7, 22, 73, 9, 64, 12]
Number of features selected:  10
After select attr:  [16, 65, 31, 49, 77, 78, 82, 33, 6, 19, 72, 23, 84, 85, 40, 58, 28, 68, 38, 66, 32, 34, 56, 8, 10, 57, 14, 15, 53, 27, 55, 80, 70, 61, 51, 71, 18, 59, 13, 11, 0, 81, 3, 5, 7, 22, 73, 9, 64, 12]
Number of features selected:  50
X1
弱分類器輸入資料維度 :  10
弱分類器錯誤率: 0.5117475334623757
SAMME弱分類器權重: -0.046998783183666015
After select attr:  [50, 15, 30, 64, 74, 73, 12]
Number of features selected:  7
After select attr:  [84, 85, 54, 67, 62, 16, 71, 39, 43, 27, 9, 59, 66, 40, 63, 17, 72,

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3709: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())
2021 15:26:29 INFO feature select success!
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3709: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())
2021 15:26:29 INFO feature select success!
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3709: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())
2021 15:26:29 INFO feature select success!
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3709: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warni

弱分類器錯誤率: 0.4916797489474145
SAMME弱分類器權重: 0.033284076627514314
After select attr:  [30, 54, 1, 25, 22, 3, 47, 37, 10, 8, 40, 46, 41, 70]
Number of features selected:  14
After select attr:  [29, 6, 65, 73, 27, 58, 51, 56, 7, 5, 61, 53, 81, 16, 19, 50, 32, 82, 4, 2, 44, 55, 18, 36, 52, 31, 35, 45, 21, 67, 12, 77, 78, 14, 30, 54, 1, 25, 22, 3, 47, 37, 10, 8, 40, 46, 41, 70]
Number of features selected:  48
X1
弱分類器輸入資料維度 :  14
弱分類器錯誤率: 0.49086638658821
SAMME弱分類器權重: 0.03653851820744636
After select attr:  [61, 47, 54, 39, 64, 58, 26, 84, 85, 4, 2]
Number of features selected:  11
After select attr:  [46, 41, 17, 67, 33, 34, 6, 57, 0, 59, 42, 15, 38, 30, 50, 31, 45, 72, 7, 5, 74, 35, 9, 63, 3, 1, 36, 22, 48, 20, 62, 49, 40, 12, 14, 25, 61, 47, 54, 39, 64, 58, 26, 84, 85, 4, 2]
Number of features selected:  47
X1
弱分類器輸入資料維度 :  11
弱分類器錯誤率: 0.5078310816486368
SAMME弱分類器權重: -0.03132688829243914


KeyboardInterrupt: 

In [ ]:
# from sklearn.ensemble import AdaBoostClassifierZe
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import numpy as np


#交叉驗證，分層隨機抽樣
from sklearn.model_selection import StratifiedKFold
import random as rd
rand = rd.randint(1, 10) #產生交叉驗證抽樣隨機種子
skf = StratifiedKFold(n_splits=10, random_state = rand, shuffle=True)

for train_index, test_index in skf.split(x_, y_):
    
    ans_auc = [] #儲存每次交叉驗證的AUC
    ans_f1 = [] #儲存每次交叉驗證的f1
    #print("TRAIN:", train_index, "TEST:", test_index,'\n')
    x_train, x_test = x_.iloc[train_index], x_.iloc[test_index]
    y_train, y_test = y_.iloc[train_index], y_.iloc[test_index]

    x_train = pd.DataFrame(x_train)
    y_train = pd.DataFrame(y_train)

    x_test = pd.DataFrame(x_test)
    y_test = pd.DataFrame(y_test)   



    clf = AdaBoostClassifier(base_estimator=GaussianNB(), n_estimators=100, random_state=0, algorithm='SAMME')
#             clf = GaussianNB()
#         clf = AdaBoostClassifierZe(base_estimator=GaussianNB(), n_estimators=100, random_state=0, algorithm='SAMME.R', fs_enable=fsMethod)
#             clf = AdaBoostClassifierZe(base_estimator=SVC(probability=True), n_estimators=100, random_state=0, algorithm='SAMME', estimator_error_calc=calcMethod, fs_enable=True)
    clf = clf.fit(x_train, y_train)

    clf.fit(x_train, y_train)

    ans_f1.append(get_f1(clf,x_test,y_test)) #取得用測試資料測試的f1
    ans_auc.append(get_auc(clf,x_test,y_test)) #取得用測試資料測試的AUC

    print('ans_auc : ', ans_auc)
    print('ans_f1 : ', ans_f1)


In [ ]:
ans_auc

In [ ]:
print(np.mean(ans_auc))

In [ ]:
ans_f1

In [ ]:
print(np.mean(ans_f1))